# 1. Linear Regression on Fish Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext version_information

In [ ]:
# The Fish Data Set
# See example 2 from https://stats.idre.ucla.edu/r/dae/zip/ 
#"nofish","livebait","camper","persons","child","xb","zg","count"
import os
from urllib.request import urlretrieve
if not os.path.isfile('fishing.npz'):
    print("Downloading")
    urlretrieve('http://www-home.htwg-konstanz.de/~oduerr/data/fishing.npz',filename = 'fishing.npz')
d = np.load('fishing.npz')
Xt = d['Xt'] #"livebait","camper","persons","child"
Xte = d['Xte']
yt = d['yt']
yte = d['yte']
pd.DataFrame(Xt[0:2])

### a) Do a linear regression by creating a design matrix with the intercept term and use the fomulae given in the lecture to determine the coefficients on the training set.

In [ ]:
intercept_column = np.ones((Xt.shape[0], 1))
Xt_with_intercept = np.concatenate((intercept_column, Xt), axis=1)

# w = (X^TX)^-1X^Ty
XTX = np.matmul(Xt_with_intercept.T, Xt_with_intercept)
XTXI = np.linalg.inv(XTX)
XTXIXT = np.matmul(XTXI, Xt_with_intercept.T)
w = np.matmul(XTXIXT, yt)
w

### b) Repeat a) but this time with LinearRegression from sklearn.linear_model

In [ ]:
from sklearn.linear_model import LinearRegression
model_skl = LinearRegression(fit_intercept=False) # We have an extended X
model_skl.fit(Xt_with_intercept,yt)
model_skl.coef_

### c) Determine the Root Mean Square Error (RMSE) and the average negative log-likelihood (NLL) on the testset. For NLL we assume that the conditional probability distrubution (CPD) p(y|x) is given by the density of a Gaussian with constant variance phi². Estimate phi² using the variance of the residuals. Use the variance estimation with 1/N.

In [ ]:
intercept_column = np.ones((Xte.shape[0], 1))
Xte_with_intercept = np.concatenate((intercept_column, Xte), axis=1)

# Calculate predicted values on the test set
y_pred = np.matmul(Xte_with_intercept, w)
# Calculate residuals
residuals = yte - y_pred
# Calculate RMSE
rmse = np.sqrt(np.mean(residuals**2))
print(rmse)
# Estimate the variance of residuals
variance_estimation = np.var(residuals, ddof=1) # ddof=1 for unbiased estimation
print(variance_estimation)
# Calculate NLL using Gaussian density
nll = 0.5 * (np.log(2 * np.pi * variance_estimation) + np.mean(residuals**2) / variance_estimation)
print(nll)

### d) For the testset: plot the predicted mean number of fish caught (µ) against observed number of fish caught. Further include the 2.5 and 97.5 precentile of p(y|x), the conditional predictive distribution (CPD) of y for a given x. Why is a Gaussian not ideal for that kind of data?

In [ ]:
# Calculate standard deviation of residuals
residual_std = np.std(residuals)
# 2.5%
lower_percentile = y_pred - 1.96 * residual_std
# 97.5%
upper_percentile = y_pred + 1.96 * residual_std

plt.scatter(y_pred, yte, alpha=0.5)
plt.fill_between(y_pred, lower_percentile, upper_percentile, color='gray', alpha=0.2, label='95% Prediction Interval')
plt.xlabel('Predicted Mean Number of Fish Caught')
plt.ylabel('Observed Number of Fish Caught')
plt.show()

Diskrete Daten, viele niedrige und wenige sehr hohe Beobachtungen sowie viele Beobachtungen von 0 Fischen.

### e) This data is count data. Count data has only positive values and also the distribution is discrete. You cannot catch 0.5 fish and that the CPD has probability density > 0 on negative number of fish is wrong too. A Gaussian as a CPD is therefore not ideal. Now use a Poissonian as CPD. [...] Use a gradient descent approach on the NLL to find the solution for the parameters. Calculate the RMSE and the NLL on the test set and compare with c).

### f) Do the same plot as in d) but this time with a Poisson CPD. Hint you can use scipy.stats.poisson to calculate the percentiles.

In [ ]:
from scipy.stats import poisson
y_pred = np.matmul(Xte_with_intercept, w)

# 2.5%
lower_percentile = poisson.ppf(0.025, y_pred)
# 97.5%
lower_percentile = poisson.ppf(0.975, y_pred)

plt.scatter(y_pred, yte, alpha=0.5)
plt.fill_between(y_pred, lower_percentile, upper_percentile, color='gray', alpha=0.2, label='95% Prediction Interval')
plt.xlabel('Predicted Mean Number of Fish Caught')
plt.ylabel('Observed Number of Fish Caught')
plt.show()

In [ ]:
%version_information